In [ ]:
#!/usr/bin/env python

# JRK: from https://breakingcode.wordpress.com/2013/04/08/finding-connected-components-in-a-graph/

# Finding connected components in a bidirectional graph.
# By Mario Vilas (mvilas at gmail dot com)

import random
import string
import time

# The graph nodes.
class Data(object):
    def __init__(self, name):
        self.__name  = name
        self.__links = set()

    @property
    def name(self):
        return self.__name

    @property
    def links(self):
        return set(self.__links)

    def add_link(self, other):
        self.__links.add(other)
        other.__links.add(self)

# The function to look for connected components.
def connected_components(nodes):

    # List of connected components found. The order is random.
    result = []

    # Make a copy of the set, so we can modify it.
    nodes = set(nodes)

    # Iterate while we still have nodes to process.
    while nodes:

        # Get a random node and remove it from the global set.
        n = nodes.pop()

        # This set will contain the next group of nodes connected to each other.
        group = {n}

        # Build a queue with this node in it.
        queue = [n]

        # Iterate the queue.
        # When it's empty, we finished visiting a group of connected nodes.
        while queue:

            # Consume the next item from the queue.
            n = queue.pop(0)

            # Fetch the neighbors.
            neighbors = n.links

            # Remove the neighbors we already visited.
            neighbors.difference_update(group)

            # Remove the remaining nodes from the global set.
            nodes.difference_update(neighbors)

            # Add them to the group of connected nodes.
            group.update(neighbors)

            # Add them to the queue, so we visit them in the next iterations.
            queue.extend(neighbors)

        # Add the group to the list of groups.
        result.append(group)

    # Return the list of groups.
    return result

# The test code...
if __name__ == "__main__":
    
    n_nodes = 1000
    
    while True:
        try:
            graph = []
            n_nodes = n_nodes + 1000
            nodes = []
            name_length = 8
            probabiliy = 1 / n_nodes
            
            for i in range(n_nodes):
                hash = ''.join(random.choice(string.ascii_uppercase + string.digits) for _ in range(name_length))
                graph.append(Data(hash))
                
            for i in range(n_nodes):
                for j in range(n_nodes):
                    if random.uniform(0.0, 1.0) < probabiliy:
                        graph[i].add_link(graph[j])
        
            # Find all the connected components.
            number = 1
            start_time = time.time()
            for components in connected_components(graph):
                names = sorted(node.name for node in components)
                names = ", ".join(names)
                # print("Group #%i: %s" % (number, names))
                number += 1
            stopped_time = time.time() - start_time
            
            print("nodes: " + str(n_nodes) + ":%s seconds" % stopped_time)
                
        except KeyboardInterrupt:
            break

nodes: 2000:0.032729148864746094 seconds
nodes: 3000:0.06838488578796387 seconds
nodes: 4000:0.12317299842834473 seconds
nodes: 5000:0.18344688415527344 seconds
nodes: 6000:0.25771498680114746 seconds
nodes: 7000:0.32267022132873535 seconds
